In [ ]:
import pandas as pd
import numpy as np
import random as rnd
import os
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


import datetime

In [ ]:
#train_df=pd.read_csv('vitalPeriodic.csv', chunksize=1000000)
demo_df=pd.read_csv('vitalPeriodicDemo.csv')

In [ ]:
demo_df = demo_df[['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'systemicdiastolic']]

In [ ]:
demo_df[['vitalperiodicid','patientunitstayid', 'observationoffset']] = demo_df[['vitalperiodicid','patientunitstayid', 'observationoffset']].astype('int32')

In [ ]:
demo_df[['temperature','sao2', 'heartrate', 'respiration','cvp', 'systemicdiastolic']] = demo_df[['temperature','sao2', 'heartrate', 'respiration','cvp', 'systemicdiastolic']].astype('float32')

In [ ]:
demo_df['vitalperiodicid'] = demo_df['vitalperiodicid'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [ ]:
temp = demo_df.dropna(subset=['temperature'])[['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'systemicdiastolic']]

In [ ]:
demo_df = demo_df.drop(demo_df[[99 == Sum for Sum in sum([demo_df.patientunitstayid!=pid for pid in temp.patientunitstayid.unique()])]].index)

In [ ]:
dt = {}
colms = ['systemicdiastolic', 'sao2', 'heartrate', 'respiration', 'cvp']
for pid in demo_df.patientunitstayid.unique():
    for col in colms:
        dt[(pid, col)] = demo_df[demo_df['patientunitstayid']==pid][col].median()


In [ ]:
for pid in temp.patientunitstayid.unique():
    for col in colms:
        temp.loc[(temp.patientunitstayid==pid) & (temp[col].isnull()), col] = dt[(pid, col)]

temp['systemicdiastolic'] = temp['systemicdiastolic'].fillna(demo_df['systemicdiastolic'].median())
temp['sao2'] = temp['sao2'].fillna(demo_df['sao2'].median())
temp['heartrate'] = temp['heartrate'].fillna(demo_df['heartrate'].median())
temp['respiration'] = temp['respiration'].fillna(demo_df['respiration'].median())
temp['cvp'] = temp['cvp'].fillna(demo_df['cvp'].median())

In [ ]:
for pid in demo_df.patientunitstayid.unique():
    for col in colms:
        demo_df.loc[(demo_df.patientunitstayid==pid) & (demo_df[col].isnull()), col] = dt[(pid, col)]

demo_df['systemicdiastolic'] = demo_df['systemicdiastolic'].fillna(demo_df['systemicdiastolic'].median())
demo_df['sao2'] = demo_df['sao2'].fillna(demo_df['sao2'].median())
demo_df['heartrate'] = demo_df['heartrate'].fillna(demo_df['heartrate'].median())
demo_df['respiration'] = demo_df['respiration'].fillna(demo_df['respiration'].median())
demo_df['cvp'] = demo_df['cvp'].fillna(demo_df['cvp'].median())

In [ ]:
col_names =  demo_df.columns
sorted_df  = pd.DataFrame(columns = col_names)
for pid in demo_df.patientunitstayid.unique():
    sorted_df = sorted_df.append(demo_df[demo_df.patientunitstayid==pid].sort_values(by='vitalperiodicid'))

In [ ]:
col_names =  temp.columns
df  = pd.DataFrame(columns = col_names)
for pid in temp.patientunitstayid.unique():
    df = df.append(temp[temp.patientunitstayid==pid].sort_values(by='vitalperiodicid'))

In [ ]:
sorted_df.head(10)

In [ ]:
df.head()

In [ ]:
sorted_df.info()

In [ ]:
sorted_df.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
def feature_fun(col, df):
    df[col].diff()
    df[col].fillna(0, inplace=True)
    df[col] = df[col].apply(lambda x: x*x)
    rms_diff = (sum(df[col])/len(df))**0.5
    return df[col].std(), df[col].kurtosis(), df[col].skew(), df[col].mean(), df[col].min(), df[col].max(), rms_diff

In [ ]:
df=pd.read_csv('df_processed.csv')

#You can skip other two boxes as they lot of time, the processed df has already been created

In [ ]:
temperature_list = sorted_df[sorted_df['temperature'].notnull()].index.tolist()
i=0
prev_idx = 0
pid = sorted_df.iloc[0]['patientunitstayid']
prev_pid = sorted_df.iloc[0]['patientunitstayid']
dct = {}

for col in colm:
    dct[col] = []

for idx in temperature_list:   
    pid = sorted_df.iloc[idx]['patientunitstayid']
    if not pid==prev_pid:
        prev_pid = sorted_df.iloc[idx]['patientunitstayid']
        prev_idx = sorted_df.index[sorted_df['patientunitstayid'] == pid].tolist()[0]
    while sorted_df.iloc[idx]['vitalperiodicid'] - sorted_df.iloc[prev_idx]['vitalperiodicid'] > datetime.timedelta(hours=6):
        prev_idx+=1
    for col in colm:
        dct[col].append(feature_fun(col, sorted_df[prev_idx:idx+1]))
    i+=1

In [ ]:
colm = ['sao2', 'heartrate', 'respiration', 'cvp', 'systemicdiastolic']
for col in colm:
    df[col+'_std'] = pd.Series(np.array(dct[col])[:,0], index=df.index)
    df[col+'_kurtosis'] = pd.Series(np.array(dct[col])[:,1], index=df.index)
    df[col+'_skewness'] = pd.Series(np.array(dct[col])[:,2], index=df.index)
    df[col+'_mean'] = pd.Series(np.array(dct[col])[:,3], index=df.index)
    df[col+'_minimum'] = pd.Series(np.array(dct[col])[:,4], index=df.index)
    df[col+'_maximum'] = pd.Series(np.array(dct[col])[:,5], index=df.index)
    df[col+'_rms_diff'] = pd.Series(np.array(dct[col])[:,6], index=df.index)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# kurtosis need atleast 4 data point otherwise till will give nan value which is not usefull
df = df.dropna()

In [ ]:
df.drop(['patientunitstayid'],axis=1, inplace = True)
df.drop(['vitalperiodicid'],axis=1, inplace = True)
df.drop(['observationoffset'],axis=1,  inplace = True)

In [ ]:

labels = df['temperature'].apply( lambda x: 1 if x>=38.0 else 0)
train_df = df.drop(['temperature'],axis=1)


In [ ]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


In [ ]:
temp, X_cv, label, y_cv = train_test_split(train_df, labels, test_size=0.1, random_state=42)

clf = LogisticRegression(C=0.15)
clf = clf.fit(temp, label)
clf.score(X_cv, y_cv)
print(classification_report(y_cv, clf.predict(X_cv)))
print('F1 score:', f1_score(y_cv, clf.predict(X_cv)))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
scores = cross_val_score(clf, train_df, labels,cv=5)
print(scores)

In [ ]:
for pid in temp.patientunitstayid.unique():
    dates = matplotlib.dates.date2num(temp[temp['patientunitstayid']==pid]['vitalperiodicid'][:])
    plt.figure(figsize=(20,10))
    plt.plot_date(dates, temp[temp['patientunitstayid']==pid]['temperature'][:])